<a href="https://colab.research.google.com/github/urielmun/capstone-lab/blob/main/script_generater.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import os
import torch
from tqdm import tqdm
from itertools import islice
import json
from transformers import pipeline

In [2]:
# Load Model
try:
    generator = pipeline(
        "text-generation",
        model="mistralai/Mistral-7B-Instruct-v0.2",
        device=0 if torch.cuda.is_available() else "cpu", # GPU가 있으면 0번 GPU, 없으면 CPU
        torch_dtype=torch.bfloat16
    )
except ImportError:
    print("device_map='auto'를 사용하려면 'accelerate' 라이브G러리가 필요합니다.")
    print("pip install accelerate bitsandbytes")
    # 'accelerate' 있는 로드 코드
    generator = pipeline(
        "text-generation",
        model="mistralai/Mistral-7B-Instruct-v0.2", ## 모델 조사
        device_map="auto",
        torch_dtype=torch.bfloat16 # 메모리 효율을 위해 추가 (선택 사항)
    )

if generator.tokenizer.pad_token_id is None:
    generator.tokenizer.pad_token_id = generator.model.config.eos_token_id
    print(f"pad_token_id가 None이므로 eos_token_id({generator.tokenizer.eos_token_id})로 설정합니다.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cuda:0


pad_token_id가 None이므로 eos_token_id(2)로 설정합니다.


In [3]:
# Prompt Template
prompt_template="""
# Role:
You are a professional screenwriter who reinterpret literary works as a visual medium. Your mission is to adapt the [original text] below into a short video script that is 3 minutes long.

# Objective:
Captures the core emotions, events, and moods of [original text] to create audiovisually attractive three-minute video scripts.

# Key Guidelines:
1. **3 mins (approximately 2-3 pages): ** Be very concise so your script doesn't exceed 3 mins. Don't try to contain everything in the text, just focus on the emotional changes of [key events] and [key people].
2. **"Show, Don't Tell": * Translate the original description or the character's inner monologue into visual and auditory elements such as conversation, behavior, facial expression, and background description.
3. **Genre and vibe:** [Enter your genre or mood here. Example: Thriller, romantic, mystery, calm, urgent] Keep your tone consistent.
4. **Applies Act 3 structure:** Short video but should have a clear 'start-middle-end' structure.
    * **Act 1 (approximately 45 seconds): ** Background and character introduction, intriguing (Inciting Incident)
    * **Act 2 (approximately 90 seconds): ** Conflict intensifies or emotions rise (Rising Action / Confrontation)
    * **Act 3 (approximately 45 seconds): ** Peak and succinct ending (Climax / Resolution)

# Script format:
Please make sure to comply with the following format.

* "SCENE [Number]. [Location] - [Time of the day/night]"
* 'VIDEO:' (What the camera sees, the person's actions and facial expressions, and background explanation)
* 'AUDIO:' (character's lines, narration (V.O.), sound effects (SFX), background music (BGM))

---

# [Special instructions]
* **Narrative (V.O.) Process:** [Choose from options: 1. Minimize narration and express everything visually. / 2. Use part of the original text as an emotional narration (V.O.)]
* **Focus:** Focus on [the feelings of a particular person or a particular prop/place] you want to emphasize.

---

# [Original text]

[Put the original content of the book you want to adapt here]

---

Based on all the instructions above, please adapt [original text] into a 3-minute video script.
"""

In [4]:
#process data
def generate_response(
    generator_pipeline,
    prompt_template: str,
    input_text: str,
    max_new_tokens: int = 2048,
    temperature: float = 0.7
) -> str:

    try:
        # 간단한 플레이스홀더 [텍스트]를 가정합니다.
        # 실제 템플릿의 플레이스홀더로 변경하세요.
        final_prompt = prompt_template.replace("[여기에 각색하고 싶은 책의 원본 내용을 붙여넣으세요.]", input_text)
    except Exception as e:
        print(f"프롬프트 구성 중 오류 발생: {e}")
        return ""

    # 2. Mistral-Instruct 모델 형식에 맞게 메시지 리스트 구성
    messages = [
        {"role": "user", "content": final_prompt}
    ]

    print("--- 모델에 프롬프트 전송 시작 ---")

    # 3. 파이프라인(모델) 호출
    outputs = generator_pipeline(
        messages,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,  # Temperature를 사용하려면 do_sample=True가 필요
        pad_token_id=generator_pipeline.tokenizer.eos_token_id
    )
    print("--- 모델 응답 수신 완료 ---")

    # 4. 결과 추출
    # 'pipeline'은 'messages' 리스트를 입력으로 받으면,
    # 응답이 추가된 'messages' 리스트를 반환합니다.
    # outputs[0]['generated_text']는 대화 내역 전체(list)입니다.
    # 우리는 그중 마지막 응답(assistant의 응답)의 'content'만 필요합니다.

    try:
        generated_chat_history = outputs[0]['generated_text']
        assistant_reply = generated_chat_history[-1]['content']
        return assistant_reply.strip()
    except (IndexError, KeyError, TypeError) as e:
        print(f"모델 출력에서 응답을 추출하는 데 실패했습니다: {e}")
        print(f"전체 출력: {outputs}")
        return ""

In [ ]:
# Input Data - file version
'''
```input json file format
record = {
            "book_title": book_title,
            "author": author,
            "scenes": scenes
        }
        ```

input_filepath="library/1.json"
with open(input_filepath,"r",encoding="utf-8")as input:
    record=json.load(line.strip())
    input_context=record.get("context","")
print("INPUT_CONTEXT:",input_context)
print("------------------------------")
'''


In [7]:
#The little prince - chapter3
input_context="""It took me a long time to understand where he came from. The little prince,
who asked me so many questions, never seemed to hear the ones I asked him.
It was things he said quite at random that, bit by bit, explained everything.
For instance, when he first caught sight of my airplane (I won't draw my
airplane; that would be much too complicated for me) he asked:
"What's that thing over there?"
"It's not a thing. It flies. It's an airplane. My airplane."
And I was proud to tell him I could fly. Then he exclaimed:
"What! You fell out of the sky?"
"Yes," I said modestly.
"Oh! That's funny..." And the little prince broke into a lovely peal of
laughter, which annoyed me a good deal. I like my misfortunes to be taken
seriously. Then he added, "So you fell out of the sky, too. What planet are you
from?"
That was when I had the first clue to the mystery of his presence, and I
questioned him sharply.
"Do you come from another planet?"
But he made no answer. He shook his head a little, still staring at my
airplane.
"Of course, that couldn't have brought you from very far..."
And he fell into a reverie that lasted a long while. Then, taking my sheep
out of his pocket, he plunged into contemplation of his treasure.
You can imagine how intrigued I was by this hint about "other planets." I
tried to learn more: "Where do you come from, little fellow? Where is this
'where I live' of yours? Where will you be taking my sheep?"
After a thoughtful silence he answered, "The good thing about the crate
you've given me is that he can use it for a house after dark."
"Of course. And if you're good, I'll give you a rope to tie him up during the
day. And a stake to tie him to."
This proposition seemed to shock the little prince. "Tie him up? What a
funny idea!"
"But if you don't tie him up, he'll wander off somewhere and get lost."
My friend burst out laughing again.
"Where could he go?"
"Anywhere. Straight ahead..."
Then the little prince remarked quite seriously, "Even if he did,
everything's so small where I live!" And he added, perhaps a little sadly,
"Straight ahead, you can't go very far."""

In [8]:
generated_script = generate_response(
        generator,
        prompt_template,
        input_context,
        max_new_tokens=2000, # 3분 스크립트는 길 수 있으므로 넉넉하게 설정
        temperature=0.7
    )

# 4. 결과 출력
print("\n--- 🤖 생성된 비디오 스크립트 ---")
print(generated_script)

--- 모델에 프롬프트 전송 시작 ---
--- 모델 응답 수신 완료 ---

--- 🤖 생성된 비디오 스크립트 ---
Title: The Forgotten Melody

Genre: Calm, introspective

---

SCENE 1. A quaint town - Sunset

VIDEO:
The sun sets over a picturesque town, as children play in the streets, and the aroma of dinner wafts from open windows. Our protagonist, old Mr. Johnson, sits alone on a bench, a forgotten violin in his lap.

AUDIO:
(Softly playing in the background) Violin melody from Mr. Johnson's past

---

SCENE 2. Old Mr. Johnson's house - Interior, night

VIDEO:
Mr. Johnson enters his home, filled with memories of his deceased wife. He walks to the attic, where dusty boxes filled with mementos sit, untouched. He finds a chest, covered in cobwebs, and opens it to reveal his old violin.

AUDIO:
(V.O.) Mr. Johnson: (sighs) How long has it been since I last held this?

---
SCENE 3. Park - Morning

VIDEO:
The following morning, Mr. Johnson arrives at the park, sitting under a tree. He takes out the violin and starts to play the melody.

In [9]:
# Save Output Data
data_to_save = {
        "original_text": input_context.strip(),
        "generated_script": generated_script
    }

output_filename = "generated_script.json"

# 5-3. 파일 쓰기
try:
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(data_to_save, f, ensure_ascii=False, indent=4)

    print(f"\n Save Sucess'{output_filename}'")

except Exception as e:
    print(f"\nError: {e} ---")


 Save Sucess'generated_script.json'
